## 독버섯과 언어
1. 전처리 - 나라별 알파벳 패턴 분석
2. 학습 - 교차검증 적용

In [4]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
import os

In [9]:

df = pd.read_csv('../DATA_WOO/mushroom.csv')
lang = pd.read_csv('../DATA_WOO/lang/lang/train/')

PermissionError: [Errno 13] Permission denied: '../DATA_WOO/lang/lang/train/'

In [6]:
os.listdir('../DATA_WOO/lang/lang/train/')

['en-1.txt',
 'en-2.txt',
 'en-3.txt',
 'en-4.txt',
 'en-5.txt',
 'fr-10.txt',
 'fr-6.txt',
 'fr-7.txt',
 'fr-8.txt',
 'fr-9.txt',
 'id-11.txt',
 'id-12.txt',
 'id-13.txt',
 'id-14.txt',
 'id-15.txt',
 'tl-16.txt',
 'tl-17.txt',
 'tl-18.txt',
 'tl-19.txt',
 'tl-20.txt']

In [2]:
from collections import Counter

def ft(FOLDER, FILENAME):
    alpha_count = np.zeros(26)
    with open(f'../DATA_WOO/lang/lang/{FOLDER}/{FILENAME}' ,mode='r' ,encoding='utf-8') as f:
        line = f.readlines()
        for i in line:
            dict = Counter(i.lower())
            alpha_list = [chr(i) for i in range(97,123)]
            for idx, alphabet in enumerate(alpha_list):
                alpha_count[idx] += dict[alphabet]
    pd.Series(alpha_count).to_csv(FILENAME)

In [55]:
for i in os.listdir('../DATA_WOO/lang/lang/train/'):
    ft('train', i)

In [64]:
df = pd.DataFrame()
for i in os.listdir('./train/'):
    df = pd.concat([pd.read_csv('./train/' + i, index_col=0),df], axis=1)

In [80]:
df.columns = pd.Series(os.listdir('./train/')).str.split('.', expand=True).iloc[:,0]
df.index = [chr(i) for i in range(97,123)]
df = df.T 
df = df.div(df.sum(axis=1), axis=0)
df_label = pd.Series(df.index).str.split('-', expand=True)[0]

### test set 전처리

In [108]:
for i in os.listdir('../DATA_WOO/lang/lang/test/'):
    ft('test', i)

In [109]:
dg = pd.DataFrame()
for i in os.listdir('./test/'):
    dg = pd.concat([pd.read_csv('./test/' + i, index_col=0),dg], axis=1)

In [111]:
dg.columns = pd.Series(os.listdir('./test/')).str.split('.', expand=True).iloc[:,0]
dg.index = [chr(i) for i in range(97,123)]
dg = dg.T 
dg = dg.div(dg.sum(axis=1), axis=0)
dg_label = pd.Series(dg.index).str.split('-', expand=True)[0]

- df, df_label, dg, dg_label
- train_data, train_target, test_data, test_target 에 각각 대응

## 모델링 작업

In [ ]:
df = df.to_numpy()
dg = dg.to_numpy()

In [8]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_validate
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier


## 로지스틱

In [126]:
lg = LogisticRegression()
lg.fit(df,df_label)

LogisticRegression()

In [127]:
print(lg.score(df,df_label))
print(lg.score(dg,dg_label))

0.85
0.75


In [129]:
result= cross_validate(lg, df, df_label, return_train_score=True)
print(np.mean(result['train_score']))
print(np.mean(result['test_score']))

0.825
0.8


## KNN

In [161]:
knn = KNeighborsClassifier()
knn.fit(df,df_label)
print(knn.score(df,df_label))
print(knn.score(dg,dg_label))

0.85
0.875


In [162]:
result= cross_validate(knn, df, df_label, return_train_score=True)
print(np.mean(result['train_score']))
print(np.mean(result['test_score']))

0.9
0.9


## SVC

In [156]:
sv = SVC()
sv.fit(df,df_label)
print(sv.score(df,df_label))
print(sv.score(dg,dg_label))

1.0
1.0


In [157]:
result= cross_validate(sv, df, df_label, return_train_score=True)
print(np.mean(result['train_score']))
print(np.mean(result['test_score']))

1.0
1.0


## decision tree

In [146]:
de = DecisionTreeClassifier()
de.fit(df,df_label)
print(de.score(df,df_label))
print(de.score(dg,dg_label))

1.0
0.75


In [147]:
result= cross_validate(de, df, df_label, return_train_score=True)
print(np.mean(result['train_score']))
print(np.mean(result['test_score']))

1.0
0.7


## RandomForest

In [138]:
rf = RandomForestClassifier()
rf.fit(df,df_label)
print(rf.score(df,df_label))
print(rf.score(dg,dg_label))

1.0
1.0


In [144]:
result= cross_validate(rf, df, df_label, return_train_score=True)
print(np.mean(result['train_score']))
print(np.mean(result['test_score']))

1.0
1.0


## 실전 test
- https://fr.wikipedia.org/wiki/Wikip%C3%A9dia:Accueil_principal  -> 프랑스어 사이트

In [149]:
ft('test','wiki.txt')

In [5]:
ft('test','indonesia.txt')

In [6]:
dh = pd.read_csv('./test/' + 'indonesia.txt', index_col=0)
dh.index = [chr(i) for i in range(97,123)]
dh = dh.T 
dh = dh.div(dh.sum(axis=1), axis=0)
dh

,a,b,c,d,e,f,g,h,i,j,...,q,r,s,t,u,v,w,x,y,z
0,0.190526,0.026842,0.000526,0.033684,0.07,0.003158,0.025789,0.019474,0.083158,0.013158,...,0.001579,0.054211,0.048947,0.051579,0.060526,0.001053,0.006842,0.0,0.017895,0.0


In [7]:
print(rf.predict(dh.to_numpy()))
print(sv.predict(dh.to_numpy()))
print(de.predict(dh.to_numpy()))
print(knn.predict(dh.to_numpy()))
print(lg.predict(dh.to_numpy()))

NameError: name 'rf' is not defined

# 결론
|모델|train_score|test_score|
|---|---|---|
|로지스틱|0.85|0.75|
|로지스틱(교차)|0.825|0.8|
|KNN|0.85|0.875|
|KNN(교차)|0.9|0.9|
|SVM|1.0|1.0|
|SVM(교차)|1.0|1.0|
|RandomForest|1.0|1.0|
|RandomForest(교차)|1.0|1.0|
|결정트리|1.0|0.75|
|결정트리(교차)|1.0|0.7|


### 실전에 강한건 KNN
